In [ ]:
import sys  
sys.path.insert(0, '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/')
import numpy as np
path_dir = r"/Users/../Volumes/Affiliations/"

In [ ]:
re

In [ ]:
import pickle as pkl
import dask
import dask.dataframe as dd

In [ ]:
import sys  
sys.path.insert(0, '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/')
import numpy as np
import src.utils
from src.utils import import_dta, import_tracks_dta, successful_creators_followers,\
gen_active_relations, get_fan_interactions_per_week, calculate_avg_monthly_valence,\
gen_active_relations_prob, get_fan_interactions_per_week_prob, stripplot_prob,\
reaction_probability, follower_list, filter_quantile, sample_creators_music,\
gen_outbound_creators
import datetime
from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mstats
pd.plotting.register_matplotlib_converters()

import src.probability_utils
from src.probability_utils import volume_analysis, valence_analysis
import math
import matplotlib.ticker as plticker
from scipy.interpolate import make_interp_spline, BSpline
import seaborn as sns
from numpy import random
from tabulate import tabulate
from IPython.display import display
from tqdm import tqdm
import os

In [ ]:
def import_dta(path_dir, path_or_buffer):
    """Imports and parsers dta data, string timestamp is converted to datatime.datetime format"""
    data = pd.read_stata(os.sep.join([path_dir, path_or_buffer]))
    data["created_at"] = pd.to_datetime(data["created_at"])
    print("%" * 10, path_or_buffer, "%" * 10)
    print(data.shape)
    return data

In [ ]:
reader=pd.read_stata(os.sep.join([r"/Users/../Volumes/Affiliations/", "12sample_1st_deg_affiliations_sent_00.dta"]),chunksize=1000000)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_0_25 = pd.DataFrame()
df_26_50 = pd.DataFrame()
df_51_75 = pd.DataFrame()
df_76_100 = pd.DataFrame()
df_101_130 = pd.DataFrame()
df_131_160 = pd.DataFrame()

i=0
for itm in reader:
    i = i+1
    print(i)
    if i <26:
        df_0_25 = df_0_25.append(itm)
    elif i < 51:
        df_26_50 = df_26_50.append(itm)
    elif i < 76:
        df_51_75 = df_51_75.append(itm)
    elif i < 100:
        df_76_100 = df_76_100.append(itm)
    elif i < 130:
        df_101_130 = df_101_130.append(itm)
    else:
        df_131_160 = df_131_160.append(itm)

In [ ]:
df_total = pd.concat([df_0_25,df_26_50,df_51_75,df_76_100,df_101_130,df_131_160])

In [ ]:
outdegree = df_total.groupby('sender_id').agg(following=('contact_id', pd.Series.nunique))

In [ ]:
path_dir = r"/Users/../Volumes/Raw/"
user_info_2 = import_dta(path_dir, "12sample_1st_deg_user_infos.dta");
user_info = import_dta(path_dir, "12sample_user_infos.dta");

In [ ]:
outdegree_info = user_info_2.merge(outdegree, left_on = 'user_id', right_on = 'sender_id', how= 'left')
outdegree_info.loc[outdegree_info.following.isnull(), 'following'] = 0 

In [ ]:
plt.figure(figsize=(12, 8))
sns.histplot(data=outdegree_info, x="following", bins=100, stat = 'density')\
.set(title = 'Distribution of out-degree: density of the number of follows sent.', xlabel = 'Follows Sent');

In [ ]:
#plt.figure(figsize=(12, 8))
#sns.boxplot(data=outdegree_info, y="following").set(title = Distribution)

In [ ]:
round(outdegree_info.following.describe(percentiles = [.1, .2, .3, .4, .5, .6, .7, .8, .9]),2)

In [ ]:
outdegree_info.following.value_counts()

In [ ]:
outdegree.shape

In [ ]:
df_list = [df_0_25,df_26_50,df_51_75,df_76_100,df_101_130,df_131_160]

In [ ]:
for i, df in enumerate(df_list):
    df.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/{}.pkl'.format(i))

# Indegree

In [ ]:
def break_big_file(path, file, index_col = None, columns = None):
    reader = pd.read_stata(os.sep.join([path, file]), chunksize=20000000, index_col=index_col, columns=columns)
    return reader

In [ ]:
def gen_agg_data(reader, j):
    i=0
    d = {}
    for itm in tqdm(reader):
        i = i+1
        d[str(i)] = itm.reset_index()
        d[str(i)]['created_at'] = pd.to_datetime(d[str(i)]['created_at']).dt.floor('d')
        d[str(i)] = d[str(i)].groupby([d[str(i)].contact_id, 'created_at'], as_index = False).size()

    print('file ended')
    df = pd.concat(d.values(), ignore_index=True)
    df = df.groupby([df.contact_id, 'created_at'], as_index = False).size()
    #df = dd.from_pandas(df)
    df.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/df{}.pkl'.format(j))

In [ ]:
path = r"/Users/../Volumes/Affilitations_received/"

for j in range(0, 10):
    file = "12sample_1st_deg_affiliations_received_0{}.dta".format(j)
    gen_agg_data(break_big_file(path, file, 'contact_id', ['contact_id', 'created_at']),j, path2)

In [ ]:
#preprocess to get rid of all infor before 2013-...

In [ ]:
def import_indegree_dask():
    df = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/df0.pkl')
    df1 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/df1.pkl')
    df2 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/df2.pkl')
    df3 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/df3.pkl')
    df4 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/df4.pkl')
    df5 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/df5.pkl')
    df6 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/df6.pkl')
    df7 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/df7.pkl')
    df8 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/df8.pkl')
    df9 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/df9.pkl')
    
    ddf = dd.from_pandas(df, npartitions = 3)
    ddf1 = dd.from_pandas(df1, npartitions = 3)
    ddf2 = dd.from_pandas(df2, npartitions = 3)
    ddf3 = dd.from_pandas(df3, npartitions = 3)
    ddf4 = dd.from_pandas(df4, npartitions = 3)
    ddf5 = dd.from_pandas(df5, npartitions = 3)
    ddf6 = dd.from_pandas(df6, npartitions = 3)
    ddf7 = dd.from_pandas(df7, npartitions = 3)
    ddf8 = dd.from_pandas(df8, npartitions = 3)
    ddf9 = dd.from_pandas(df9, npartitions = 3)
    concatdf = dd.multi.concat([ddf,ddf1,ddf2,ddf3,ddf4,ddf5,ddf6,ddf7,ddf8,ddf9])
    concatdf = concatdf.compute()
    return concatdf

In [ ]:
len(concatdf)

In [ ]:
path_dir = r"/Users/../Volumes/Raw/"
tracks = import_tracks_dta(path_dir, "12sample_tracks.dta");

In [ ]:
mask = (tracks.track_available == 1) & (tracks.public == 't')
creator_ids = tracks[mask].user_id.unique()

creators = tracks[(tracks.track_available == 1) & (tracks.public == 't')]

In [ ]:
path_dir = r"/Users/../Volumes/Raw/"

shares_received = import_dta(path_dir, "12sample_reposts_received.dta");
follows_received = import_dta(path_dir, "12sample_affiliations_received.dta");

In [ ]:
# List of users that share the content of at least one creators
alters_shared_id = shares_received.loc[shares_received.owner_id.isin(creators.user_id.unique())].reposter_id.unique()

In [ ]:
comments_received_35k = import_dta(path_dir, "12sample_comments_received.dta");
shares_received_35k = import_dta(path_dir, "12sample_reposts_received.dta");
likes_received_35k = import_dta(path_dir, "12sample_favoritings_received.dta");
messages_received_35k = import_dta(path_dir, "12sample_messages_received.dta");

if 'song_id' in shares_received_35k:
        shares_received_35k.drop(columns=["song_id"])
shares_received_35k = shares_received_35k[['reposter_id', "owner_id", 'created_at']]
shares_received_35k['outbound_activity'] = 'share'
shares_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

if 'track_id' in likes_received_35k:
        likes_received_35k = likes_received_35k.drop(columns=["track_id"])
likes_received_35k['outbound_activity'] = 'like'
likes_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

if 'track_id' in comments_received_35k:
        comments_received_35k = comments_received_35k.drop(columns=["track_id"])
comments_received_35k['outbound_activity'] = 'comment'
comments_received_35k.columns = ['fan_id', 'user_id', 'date_sent', 'outbound_activity']

messages_received_35k["outbound_activity"] = 'message'
messages_received_35k.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity']

user_activity_data_35k = pd.concat([shares_received_35k, likes_received_35k, comments_received_35k, messages_received_35k])

In [ ]:
# List of users that sent at least one action to creators
alters_acted_id = user_activity_data_35k.loc[user_activity_data_35k.user_id.isin(creators.user_id.unique())].fan_id.unique()


# List of users that sent at least one follow to creators
alters_follow_id = follows_received.loc[follows_received.contact_id.isin(creators.user_id.unique())].fan_id.unique()

In [ ]:
def gen_agg_data_2(reader, j, filter_list, save_path):
    i=0
    d = {}
    for itm in tqdm(reader):
        i = i+1
        d[str(i)] = itm.reset_index()
        d[str(i)] =  d[str(i)].loc[d[str(i)].contact_id.isin(filter_list)]
        d[str(i)]['created_at'] = pd.to_datetime(d[str(i)]['created_at']).dt.floor('d')

    print('file ended')
    df = pd.concat(d.values(), ignore_index=True)
    print('concatenation done')
    #df.sort_values('created_at', inplace=True)
    #print('sorting done')
    df = df.groupby(['contact_id'])['fan_id'].apply(np.array)
    print('list done')
    print(sys.getsizeof(df))
    df.to_pickle('{}/df{}.pkl'.format(save_path, j))

In [ ]:
!mkdir /Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list/user_followers/follows

In [ ]:
path = r"/Users/../Volumes/Affilitations_received/"
save_path = '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list'
f = gen_agg_data_2
#f = gen_agg_data_dask

for j in tqdm(range(0, 10)):
    file = "12sample_1st_deg_affiliations_received_0{}.dta".format(j)
    f(break_big_file(path, file, 'contact_id', ['contact_id', 'fan_id', 'created_at']),j, alters_shared_id, save_path)

In [ ]:
path = r"/Users/../Volumes/Affilitations_received/"
save_path = '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list/user_followers/actions'
f = gen_agg_data_2
#f = gen_agg_data_dask

for j in tqdm(range(0, 10)):
    file = "12sample_1st_deg_affiliations_received_0{}.dta".format(j)
    f(break_big_file(path, file, 'contact_id', ['contact_id', 'fan_id', 'created_at']),j, alters_acted_id, save_path)

In [ ]:
path = r"/Users/../Volumes/Affilitations_received/"
save_path = '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list/user_followers/follows'
f = gen_agg_data_2
#f = gen_agg_data_dask

for j in tqdm(range(0, 10)):
    file = "12sample_1st_deg_affiliations_received_0{}.dta".format(j)
    f(break_big_file(path, file, 'contact_id', ['contact_id', 'fan_id', 'created_at']),j, alters_follow_id, save_path)

In [ ]:
def import_indegree_dask_list():
    df = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list/df0.pkl')
    print('df done')
    
    df1 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list/df1.pkl')
    
    print('df1 done')
    df2 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list/df2.pkl')
    print('df2 done')
    
    df3 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list/df3.pkl')
    print('df3 done')
    
    df4 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list/df4.pkl')
    print('df4 done')
    
    df5 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list/df5.pkl')
    print('df5 done')
    
    df6 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list/df6.pkl')
    print('df6 done')
    
    df7 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list/df7.pkl')
    print('df7 done')
    
    df8 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list/df8.pkl')
    print('df8 done')
    
    df9 = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list/df9.pkl')
    print('Imports done')
    
    
    print('Dask done')
    concatdf = pd.concat([df,df1,df2,df3,df4,df5,df6,df7,df8,df9])
    return concatdf

In [ ]:
df = import_indegree_dask_list()
df = df.groupby(df.index).apply(np.array)
data = df.apply(lambda x: np.concatenate(x))

In [ ]:
data.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/alter_follower_lists.pkl')

In [ ]:
data

In [ ]:
indegree = indegree.compute()

In [ ]:
fan_id_list = indegree.fan_id.unique()

In [ ]:
sum(shares_received.loc[shares_received.owner_id.isin(creators.user_id.unique())].reposter_id.isin(fan_id_list))

In [ ]:
shares_received.loc[shares_received.owner_id.isin(creators.user_id.unique())].shape

In [ ]:
df.index.nunique()

In [ ]:
data = import_indegree_dask()

In [ ]:
data

In [ ]:
sum(shares_received.loc[shares_received.owner_id.isin(creators.user_id.unique())].reposter_id.isin(data.contact_id.unique()))

In [ ]:
shares_received.loc[shares_received.owner_id.isin(creators.user_id.unique())].reposter_id.nunique()

In [ ]:
sum(shares_received.loc[shares_received.owner_id.isin(creators.user_id.unique())].reposter_id.isin(user_info.user_id.unique()))

In [ ]:
def import_indegree_dask_list_2(path = '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_list/user_followers/follows/'):
    df = pd.read_pickle('{}df0.pkl'.format(path))
    print('df done')
    
    df1 = pd.read_pickle('{}df1.pkl'.format(path))
    
    print('df1 done')
    
    df2 = pd.read_pickle('{}df2.pkl'.format(path))
    print('df2 done')
    
    df3 = pd.read_pickle('{}df3.pkl'.format(path))
    print('df3 done')
    
    df4 = pd.read_pickle('{}df4.pkl'.format(path))
    print('df4 done')
    
    df5 = pd.read_pickle('{}df5.pkl'.format(path))
    print('df5 done')
    
    df6 = pd.read_pickle('{}df6.pkl'.format(path))
    print('df6 done')
    
    df7 = pd.read_pickle('{}df7.pkl'.format(path))
    print('df7 done')
    
    df8 = pd.read_pickle('{}df8.pkl'.format(path))
    print('df8 done')
    
    df9 = pd.read_pickle('{}df9.pkl'.format(path))
    print('Imports done')
    
    
    print('Dask done')
    concatdf = pd.concat([df,df1,df2,df3,df4,df5,df6,df7,df8,df9])
    return concatdf

In [ ]:
df = import_indegree_dask_list_2()
df = df.groupby(df.index).apply(np.array)
data = df.apply(lambda x: np.concatenate(x))

data.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/alter_follower_lists_3.pkl')

# Follows received from alters of successful creator

In [ ]:
with open("/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/followed_successful_list", "rb") as fp:   # Unpickling
     successful_list = pkl.load(fp)

In [ ]:
def gen_agg_data_3(reader, j, filter_list, save_path):
    i=0
    d = {}
    for itm in tqdm(reader):
        i = i+1
        d[str(i)] = itm.reset_index()
        d[str(i)] =  d[str(i)].loc[d[str(i)].contact_id.isin(filter_list)]
        d[str(i)]['created_at'] = pd.to_datetime(d[str(i)]['created_at']).dt.floor('d')

    print('file ended')
    df = pd.concat(d.values(), ignore_index=True)
    print('concatenation done')
    #df.sort_values('created_at', inplace=True)
    #print('sorting done')
    print('list done')
    print(sys.getsizeof(df))
    df.to_pickle('{}/df{}.pkl'.format(save_path, j))

In [ ]:
path = r"/Users/../Volumes/Affilitations_received/"
save_path = '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_df'
f = gen_agg_data_3
#f = gen_agg_data_dask

for j in tqdm(range(0, 10)):
    file = "12sample_1st_deg_affiliations_received_0{}.dta".format(j)
    f(break_big_file(path, file, 'contact_id', ['contact_id', 'fan_id', 'created_at']),j, successful_list, save_path)

In [ ]:
df = import_indegree_dask_list_2(path = '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/indegree/follower_df/')

df.to_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/Data/follows_received_by_alters_of_successful_creators.pkl')

In [ ]:
df.shape